### Adconsys Oy
Adconsys on suomalainen yritys, joka tarjoaa rakennuksien energiatehokkuutta parantavia automaatiojärjestelmiä ja niiden hallintajärjestelmiä.

## Tavoite

- Luoda ennustusmalli, jolla voidaan minimoida lämmityskustannuksia säätämällä ilmanvaihtokoneen lämmityspatterille tulevan veden lämpötilaa.

![](https://gitlab.dclabra.fi/wiki/uploads/upload_c42ae6ae3cc67623a82bba8fe61217bb.png)

- Venttiili TV1.1 on PID -säädin, joka säätää venttiiliä laskemalla ennustun- sekä mitatun lämpötilan avulla.
- Mallin opetuksessa käytetään menovedelle laskettua tavoitearvoa, sekä ilmanvaihtokoneesta, lämmönjakokeskusta ja ulkoilmasta mitattuja arvoja.

- Tällä hetkellä säädöt hoituvat ennalta lasketuilla säätöarvoilla, jotka on yleensä laskettu hieman yläkanttiin.

## Projektisuunnitelma
Tarkemmat vaatimukset ja reunaehdot löytyvät [projektisuunnitelmasta](https://gitlab.dclabra.fi/jvartiainen/proj3-team-adconsys/-/blob/master/Dokumentit/Projekti_3.pdf).

![](https://gitlab.dclabra.fi/wiki/uploads/upload_c3f3367eae12108794942a277100f299.png)

## Datan muokkaaminen

#### Tiedostot
Data oli aluksi kymmenessä .xml-tiedostossa. Tiedostot muutettiin .csv-muotoon, jotta ne on helpompi ottaa käsittelyyn Pandasilla.

![](https://gitlab.dclabra.fi/wiki/uploads/upload_2a7ddeff26fe667146e09aa011f4cb68.png)

|Tiedosto|Koko (MiB)|Rivimääärä|Alku|Loppu|
|---|---|---|---|---|
|IV_TE3A_Meno_M|19.3|533,439|18-May-20 8:30 PM EEST|07-May-21 7:57 AM EEST|
|IV_TE3A_Menovesi_LaskAs|0.8|21,752|03-Apr-19 8:51 AM EEST|07-May-21 7:36 AM EEST|
|TK1_LTO_S|3.9|108,333|16-May-19 3:16 PM EEST|06-May-21 4:03 AM EEST|
|TK1_TE1_2_PatterinPaluuvesi_M|3.8|106,585|04-Feb-20 10:40 PM EET|07-May-21 6:14 AM EEST|
|TK1_TE1_3_Tulo_M|2.1|59,388|17-Dec-19 10:06 AM EET|06-May-21 9:59 PM EEST|
|TK1_TE1_3_Tuloilma_LaskettuAs|0.24|6,576|03-Apr-19 8:51 AM EEST|03-May-21 4:39 PM EEST|
|TK1_TE1_4_Poisto_M|0.19|5,021|03-Apr-19 8:51 AM EEST|07-May-21 3:59 AM EEST|
|TK1_TF1_PE1_1_LaskAs|0.06|1,547|03-Apr-19 8:51 AM EEST|07-May-21 6:00 AM EEST|
|TK1_TV1_1_Lammitys_S|3.5|97,625|23-Jan-20 9:28 PM EET|07-May-21 6:23 AM EEST|
|Ulkolampotila|1.1|31,521|03-Apr-19 8:51 AM EEST|07-May-21 7:36 AM EES|

#### Yhdistäminen

Kaikki kymmenen tiedostoa yhdistettiin yhdeksi dataframeksi timestampin avulla. Timestampin muokattiin datetime-muotoon, joka huomioi kesä- ja talviajan, sekä muutti datan 24h-muotoon.
Yhdistämisessä jokaisen tiedoston kaikki tiedostot lisättiin dataframeen, koska jokaisessa arvoja on eri frekvenssillä.
Tällä tavoin virheellinen data saadaan poistettua samanaikaisesti kaikista tiedostoista.


![](https://gitlab.dclabra.fi/wiki/uploads/upload_3e95bb5629d7fbe4536da3787a860f4c.png)

Osassa tidostoista dataa oli jo vuodelta 2019. Yhdessä tiedostossa dataa oli vasta toukokuulta 2020, jonka takia yhdistetyssa dataframessa oli paljon NaN-arvoja ennen arvoa 2020-05-18 20:30:00, NaN-arvot poistettiin, jonka jälkeen dataframe näyttää seuraavalta:

![](https://gitlab.dclabra.fi/wiki/uploads/upload_021971e959c24137312f5fb6b2686ace.png)

value-sarakkeiden kuvaajat:

![](https://gitlab.dclabra.fi/wiki/uploads/upload_d2af3ba7a774fecd9f602ceb80ec74e6.png)

1LaskAs (koneen nopeus) ja TuloilmaLaskettuAs -sarakkeiden marraskuussa olevan "piikin" avulla voidaan datat yrittää synkronisoida "kohdikkain", jos se on tarpeellista.     
Nyt katsomme tuon piikin olevan selkeä outlier, ja se tullaan poistamaan datasta, sillä sen aiheuttanut ilmanvaihtokoneen pysähdys poistetaan myös datasta.

## Virheellisen datan poistaminen
#### Dataa poistettiin Adconsysiltä saadun tiedon pohjalta seuraavasti:

#### Lämpötila ja säätöarvot
Lämpötilaa mittaavat sensorit tallentavat uuden arvon aina, kun lämpötilassa tapahtuu 0.5 asteen muutos.
Säätöarvoja mittaavat sensorit tallentavat uuden arvon aina, kun säädössä tapahtuu 1% muutos.

Huomasimme, että järjestelmä on kuitenkin melko hidas, ja jos muutosta tapahtuu nopeasti, saataa kahden rivin välinen muutos olla edellä mainittuja arvoja huomattavasti suurempi. Päätimme, että emme erikseen lähde poistamaan dataa kähden rivin välisen muutoksen perusteella, koska melko helposti lähdetään poistamaan kunnollista dataa.


#### Status
Status -sarakeessa olevat arvot kertovat tallennetun mittaustuloksen "tilan". 
Fault- ja Down-arvot tarkoittavat, että systeemissä on jokin virhe, eikä mittaustulokset ole luotettavia. Nämä arvot voidaan suoraan poistaa.
Overridden-arvot ovat käsin säädettyjä arvoja. Mittaus- ja säätödataa sisältävistä tiedostoista nämä voidaan poistaa.

#### TrendFlags
TrendFlags-sarakkeen arvoissa ei ole mitään tarpeellista tietoa.

#### LTO:n arvo < 100
Tarkoittaa, että lämmöntalteenottoa ei hyödynnetä täysin. Tällöin venttiilin TV1.1 on oltava suljettuna. Menoveden lämpötilaa ei siis tarvitse ennustaa, eli <100 arvot sisältävät rivit poistetaan.

#### Datan normalisointi.
Piirrematriisi normalisoidaan, jotta treeni- testi- ja validointisetteihin jaettavat arvot ovat skaalattu yhtenäisellä datalla.     
Targettia (value_MenovesiLaskAs) ei pidä normalisoida, sillä ennustetun arvon tulee olla oikea lämpötila-arvo.

Näiden operaatioiden jälkeen data näyttää tältä:

![](https://gitlab.dclabra.fi/wiki/uploads/upload_4dd33b6a514b475e67ec4c6ba527e355.png)


Marraskuussa ollut piikki poistettiin datasta laskemalla "value_PatterinPaluuvesiM","value_PoistoM" arvojen normaalijakauma ja poistamalla arvot, jotka ovat yli kolmen "hajonnan" päästä keskiarvosta.

Syksyllä LTO on hyvin pitkälti riittänyt raitisilman lämmitykseen, jonka seurauksena lähes kaikki syksyn data on menetetty. 


Datan määrä eri operaatioiden jälkeen:     


![](https://gitlab.dclabra.fi/wiki/uploads/upload_5b3669be1b444f59a982b5532aa23c27.png)


## Optional
####  TK1_TV1_1_Lammitys_S
IV-koneen lämmityspatterin venttiilin säätöarvossa on hetkellisiä piikkejä. Adconsysiltä tuli tieto, että nämä hetkelliset piikit eivät aina välttämättä ole vihreellisiä arvoja. Halutessa piikit voidaan kuitenkin poistaa.

#### Kesäkuukausien poistaminen
Kesällä koulu on kiinni ja on lämmin, joten lämmityksestä ei tarvitse huolehtia. overridden- ja LTO < 100 -arvojen poistamisen jälkeen dataan jää joitakin satoja rivejä dataa kesä- heinä- ja elokuilta. Nämä voidaan suoraan poistaa, mutta ehkä olisi hyvä tarkistaa, voidaanko näitäkin rivejä kuitenkin hyödyntää. Esimerkiksi jos ulkolämpötilassa on tapahtunut suuria muutoksia, saattaa data olla ihan käytettävää.


Loimme toisen datasetin, jossa nämä arvot ovat poistettuna. Käytännössä näin puhdasta dataa ei taida ilmanvaihtosysteemistä saada pihalle.    
Ajattelimme, että voisi kuitenkin olla ihan mielenkiintoista verrata, saadaanko siistimmällä datalla ennustettua parempia tuloksia. 

Data näyttää tältä:    

![](https://gitlab.dclabra.fi/wiki/uploads/upload_34dd83fb5aa3da69f270998645a9db03.png)

Suorien viivojen kohdilta on kaikki data poistettu.
Poistoilman lämpötilan ja tuloilman tavoitearvot ovat hieman paremmin jakautunut, sekä lämmityspatterille saapuvan veden venttiilin arvot ovat huomattavasti selkeämmät.

#### Ylimääräisten sarakkeiden poistaminen
Datan siivoamisessa käytetyt sarakkeet voidaan poistaa, kun siivous on saatu valmiiksi. Näitä ovat kaikki muut sarakkeet paitsi value-sarakkeet. Myös LTO_S voidaan poistaa, sillä siinä on jäljellä enää vain arvoja 100.

Jäljelle jäävä data:

![](https://gitlab.dclabra.fi/wiki/uploads/upload_dbb4c675c1a03585a748f72f13045357.png)

![](https://gitlab.dclabra.fi/wiki/uploads/upload_702dcb702ebf8152357edbe7684f65b8.png)

## Datasetit

Siivottu data jaetaan treeni-, testi- ja validointisetteihin.
Kokonaisdataa on 642011 riviä. Näistä 407,175 riviä jää treenisettiin, 117,269 riviä testisettiin ja 117,567 validointisettiin.
- Treeni 63.4%
- Testi 18.3%
- Validointi 18.3%

Testisettiin erotetaan kaikki '2021-02-15 11:21:00' jälkeen oleva data.
Loppu data jaetaan treeni- ja validointisetteihin siten, että joka viides päivä erotetaan validointisettiin.

Kuvaajasta nähdään miten data on jakautunut eri setteihin.


![](https://gitlab.dclabra.fi/wiki/uploads/upload_bd2ef4050015ef146b8021916f17acc2.png)

Lopuksi dataseteistä erotetaan "IV_TE3A_Menovesi_LaskAs" -sarake tavoitevektoriksi ja loput sarakkeet piirrematriisiksi.
Piirrematriisit ja tavoitevektorit tallennetaan omiin tiedostoihin, joista ne voidaan hakea käyttöön koulutusta ja verkon arvoimista varten.    


## Menoveden lämpötilan ennustaminen

Alla muutamia ensimmäisiä neuroverkkotestejä. Lämpötilaa ennustetaan 1°C tarkkuudella.

|Optimizer|Criterion|Activation|IP L|HL 2|HL 3|output|Accuracy|Epochs|Max +virhe| Max -virhe|Keskihajonta|
|---|---|---|---|---|---|---|---|---|---|---|---|
|```Adam(lr=0.001)```   |```CrossEntropyLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.729  |3||||
|```Adam(lr=0.001)```   |```CrossEntropyLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.715  |5||||
|```Adam(lr=0.001)```   |```MultiMarginLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.736  |3||||
|```Adam(lr=0.001)```   |```MultiMarginLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.701  |5||||
|```Adam(lr=0.001)```   |```MultiMarginLoss```   |```ReLu```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128)```   |```Linear(128,67), LogSoftmax```|0.743  |7||||

Tarkkuus ei ole kovin hyvä mittari tässä, sillä se ei kerro kuinka kauakana väärin ennustettu arvo on.

![](https://gitlab.dclabra.fi/wiki/uploads/upload_7e0eed18963c69fa79189072e1a321a8.png)

Melko yksinkertaisella neuroverkolla saadaan jo ihan hyviä tuloksia. 3°C on kuitenkin  liian kaukana tavoitearvosta.

# Regressioverkot

<!-- ![](https://gitlab.dclabra.fi/wiki/uploads/upload_d89db721f0b666de0f47859ebfde3dce.png) -->

|Optimizer|Criterion|Input|Hidden 1|Hidden 2|Hidden 3|Hiden 4|output|Epochs|
|---|---|---|---|---|---|---|---|---|
|```Adam(lr=0.001)```|```MSELoss()```|```Linear(8, 512)```   |```Linear(512, 256)```   |```Linear(256, 128) ```  |```Linear(128, 64) ```  |```Linear(64, 32) ```  |```Linear(32,1)```|15|


![](https://gitlab.dclabra.fi/wiki/uploads/upload_d77edf5e091aa6eae696d5848aff010d.png)

#### ![](https://gitlab.dclabra.fi/wiki/uploads/upload_aaf52ec7197bd68b450080590ea950ee.png)

In [2]:
# pandoc --to pdf --from ipynb -o Demo.pdf  Demo.ipynb 

In [ ]:
# pandoc --to pdf --from ipynb -o Esitys.pdf  Esitys.ipynb 